In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\pakal\Downloads\chatbot\chatbot_env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("../data/")

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)

In [9]:
print(len(text_chunks))

4359


In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\pakal\AppData\Local\Temp\ipykernel_2896\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\pakal\Downloads\chatbot\chatbot_env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
embeddings.embed_documents

<bound method HuggingFaceEmbeddings.embed_documents of HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)>

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result), type(query_result))



Length 384 <class 'list'>


In [14]:
from langchain.vectorstores import Pinecone as langchain_pinecone
from pinecone import Pinecone

pinecone_api_key = ""

pc = Pinecone(api_key= pinecone_api_key)
index = pc.Index("chatbot")
vectorstore = langchain_pinecone(index, embeddings.embed_query, text_key="content")

C:\Users\pakal\AppData\Local\Temp\ipykernel_2896\2080010095.py:8: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  vectorstore = langchain_pinecone(index, embeddings.embed_query, text_key="content")
c:\Users\pakal\Downloads\chatbot\chatbot_env\Lib\site-packages\langchain_community\vectorstores\pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [15]:
vectorstore.add_documents(t for t in text_chunks)


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4685}},
 'total_vector_count': 4685}

In [19]:
index.describe_index_stats()
result = vectorstore.similarity_search(query="What is BERT?", k=4)
result

[Document(metadata={}, page_content='• It is replaced with the unique vocabulary token [MASK] .\n• It is replaced with another token from the vocabulary, randomly sampled\nbased on token unigram probabilities.\n• It is left unchanged.\nIn BERT, 15% of the input tokens in a training sequence are sampled for learning.\nOf these, 80% are replaced with [MASK] , 10% are replaced with randomly selected\ntokens, and the remaining 10% are left unchanged.\nThe MLM training objective is to predict the original inputs for each of the'),
 Document(metadata={}, page_content='BERTS CORE algorithm (Zhang et al., 2020) shown in Fig. 13.7, for example, passes\nthe reference xand the candidate ˜ xthrough BERT, computing a BERT embedding\nfor each token xiand ˜xj. Each pair of tokens (xi,˜xj)is scored by its cosinexi·˜xj\n|xi||˜xj|.\nEach token in xis matched to a token in ˜ xto compute recall, and each token in ˜ x\nis matched to a token in xto compute precision (with each token greedily matched\nto the

In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [27]:
from huggingface_hub import login
hugging_face_api = ""
login(hugging_face_api)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\pakal\.cache\huggingface\token
Login successful


In [31]:
from transformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf", model_type="llama", gpu_layers=0  )

model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf")




Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

: 

In [ ]:
import transformers
import torch
from transformers import AutoTokenizer
model_id = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
pipeline = transformers.pipeline(
    "text-generation", 
    model = model_id,
    tokenizer = tokenizer,
    torch_dtype = torch.float16,
    do_sample = True,
    top_k=10,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id,
    device_map = 'auto',
    model_kwargs = { 'max_length': 1240},
)


In [ ]:
from langchain_huggingface import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0.5})

In [ ]:
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectorstore.as_retriever(search_kwargs={'k': 10}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({"query": user_input})
#     print("Response : ", result["result"])
question = "What is the difference between attention and cross attention?"
result = qa(
    {"query": question}
)